In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/audi.csv')


In [ ]:
data

In [ ]:
data.isna().sum()

# LOOKING AT LINEAR RELATIONS

In [ ]:
data.plot(kind="scatter",x="mileage",y="price")

In [ ]:
data.plot(kind="scatter",x="tax",y="price")

In [ ]:
data.plot(kind="scatter",x="mpg",y="price")

In [ ]:
data.plot(kind="scatter",x="year",y="price")

In [ ]:
data.plot(kind="scatter",x="engineSize",y="price")

In [ ]:
data.plot(kind="scatter",x="model",y="price")

In [ ]:
data.groupby("price").mean()

# CORRELATION MATRIX

In [ ]:
def corr(dataframe,target_variable):
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10,10))
    correlation_matrix = dataframe.corr().round(2)
    sns.heatmap(data=correlation_matrix, annot=True)
    
    correlation = data.corr()[target_variable].abs().sort_values(ascending = False)
    return correlation


corr(data,"price")

**NOT THE BEST WAY TO ENCODE LABELS I KNOW**

In [ ]:
data2=data
data2=data2.drop(["year"],axis=1)
data2=data2.drop(["mileage"],axis=1)
data2=data2.drop(["tax"],axis=1)
data2=data2.drop(["mpg"],axis=1)
data2=data2.drop(["engineSize"],axis=1)
data2=data2.drop(["price"],axis=1)

from sklearn import preprocessing 
data2=data2.apply(preprocessing.LabelEncoder().fit_transform)

data=data.drop(["transmission"],axis=1)
data=data.drop(["fuelType"],axis=1)
data=data.drop(["model"],axis=1)


data=pd.concat([data,data2],axis=1)

# Having encoded the object variables we have now a better Correlation Matrix

In [ ]:
corr(data,"price")

# Feature Importance from RandomForestRegressor (i recomend it , is very useful)

In [ ]:
def feature_importance(dataframe,target_variable):
    from sklearn.ensemble import RandomForestRegressor
    import seaborn as sns
    v=12
    w=6
    considered_features=[]
    for x in dataframe.columns.values.tolist():
        if x==target_variable:
            pass
        else:
            considered_features.append(x)
    col_to_consider=[]
    for x in considered_features:
        col_to_consider.append(x)
    X= dataframe[col_to_consider]
    Y= dataframe[target_variable]
    random_forest= RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=20, min_samples_split=40).fit(X,Y)
    feature_importances_=random_forest.feature_importances_
    feature_importances=pd.DataFrame({'Feature_name':col_to_consider, 'Feature_importance':feature_importances_})
    if len(considered_features)>15:
        v=32
        w=24
    elif len(considered_features)>30:
        v=64
        w=48
    else:
        pass
    fig, ax=plt.subplots(1, figsize=(v,w))
    sns.barplot(x='Feature_name', y='Feature_importance', data=feature_importances, ax=ax)
    plt.xticks(fontsize=10, rotation=90);
    plt.yticks(fontsize=12);
    plt.xlabel('Feature name',fontsize=v)
    plt.ylabel('Feature importance',fontsize=v)
    

In [ ]:
feature_importance(data,"price")

#  As the correlation of Transmission and FuelType with the target variable (price) is too low, we wont use them

# VIF

In [ ]:
chosen_col=["year","mileage","tax","mpg","engineSize","model"]
def VIF(dataframe,chosen_col):
    from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
    from statsmodels.tools.tools import add_constant
    X=dataframe[chosen_col]
    X=add_constant(X)
    vif_data=pd.DataFrame()
    vif_data["feature"]=X.columns
    vif_data["VIF"]=[VIF(X.values, i) for i in range(len(X.columns))]
    return vif_data

In [ ]:
VIF(data,chosen_col)

# Looking for outliers

In [ ]:
def Outliers(dataframe,chosen_cols):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    numeric_col2=[]
    for x in cols:
        numeric_col2.append(x)

    fig=make_subplots(rows=1, cols=len(cols))

    for i,col in enumerate(numeric_col2):
        fig.add_trace(go.Box(y=dataframe[col].values, name=dataframe[col].name), row=1, col=i+1)

    return fig.show()

In [ ]:
cols=["year","mileage","tax","mpg","engineSize","model"]
Outliers(data,cols)

In [ ]:
data=data[~(data["year"]<2000)]
data=data[~(data["mileage"]>=168017)]
data=data[~(data["tax"]>=515)]
data=data[~(data["tax"]<45)]
data=data[~(data["mpg"]>=117.7)]
data=data[~(data["engineSize"]>=4.1)]
data=data[~(data["model"]>=22)]

# Linear Prediction

In [ ]:
def linear_model(dataframe,feature_cols,target_variable):
    from sklearn.linear_model import LinearRegression
    X=dataframe[feature_cols]
    Y=dataframe[target_variable]
    lm= LinearRegression()
    lm.fit(X,Y)
    dataframe["Predict"]=lm.predict(X)
    dataframe["RSE"]= (dataframe[target_variable] - dataframe["Predict"])**2
    SSD= sum(dataframe["RSE"])
    RSE= np.sqrt(SSD/(len(dataframe)-len(feature_cols) -1 ))
    target_mean= np.mean(dataframe[target_variable])
    error= RSE/target_mean
    print(f"The intercept is {lm.intercept_}\n")
    print(f"The coefs are: {list(zip(feature_cols,lm.coef_))}\n")
    print(f"The R2 is: {lm.score(X,Y)}\n")
    print(f"The SSD is: {SSD}\n")
    print(f"The RSE is: {RSE}\n")
    print(f"The {target_variable} mean is: {target_mean}\n")
    print(f"The error is: {error}\n")
    if len(feature_cols)==1:
        %matplotlib inline
        plt.plot(X,Y,"go")
        plt.plot(X,lm.predict(X),color="red")
        for x in feature_cols:
            plt.xlabel(x)
            plt.title(f"{target_variable} vs {x}")
        plt.ylabel(target_variable)
    else:
        pass


In [ ]:
feature_cols=["year","mileage","tax","mpg","engineSize","model"]
linear_model(data,feature_cols,"price")

# The relation between price and all the variables is not linear so we will try a cuadratic model

In [ ]:
def cuadratic_model(dataframe,feature_cols,target_variable):
    from sklearn.linear_model import LinearRegression
    X=dataframe[feature_cols]**2
    Y=dataframe[target_variable]
    lm= LinearRegression()
    lm.fit(X,Y)
    dataframe["Cuadratic_Predict"]=lm.predict(X)
    dataframe["Cuadratic_RSE"]= (dataframe[target_variable] - dataframe["Cuadratic_Predict"])**2
    SSD= sum(dataframe["Cuadratic_RSE"])
    RSE= np.sqrt(SSD/(len(dataframe)-len(feature_cols) -1 ))
    target_mean= np.mean(dataframe[target_variable])
    error= RSE/target_mean
    print(f"The intercept is {lm.intercept_}\n")
    print(f"The coefs are: {list(zip(feature_cols,lm.coef_))}\n")
    print(f"The R2 is: {lm.score(X,Y)}\n")
    print(f"The SSD is: {SSD}\n")
    print(f"The RSE is: {RSE}\n")
    print(f"The {target_variable} mean is: {target_mean}\n")
    print(f"The error is: {error}\n")
    if len(feature_cols)==1:
        %matplotlib inline
        plt.plot(X,Y,"go")
        plt.plot(X,lm.predict(X),color="red")
        for x in feature_cols:
            plt.xlabel(x)
            plt.title(f"{target_variable} vs {x}")
        plt.ylabel(target_variable)
    else:
        pass

In [ ]:
cuadratic_model(data,feature_cols,"price")

**We can see a small increase and decrease of R2 and the error**

# Now we will try a polynomial model at the same time, using different degrees to see whom  suits better

In [ ]:
def polynomial_model(dataframe,feature_cols,target_variable,degree_number):
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn import linear_model
    poly= PolynomialFeatures(degree=degree_number)
    X= poly.fit_transform(dataframe[feature_cols])
    Y= dataframe[target_variable]
    lm= linear_model.LinearRegression()
    lm.fit(X,Y)
    dataframe["Cuadratic_Linear_Predict"]=lm.predict(X)
    dataframe["Cuadratic_Linear_RSE"]= (dataframe[target_variable] - dataframe["Cuadratic_Linear_Predict"])**2
    SSD= sum(dataframe["Cuadratic_Linear_RSE"])
    RSE= np.sqrt(SSD/(len(dataframe)-len(feature_cols) -1 ))
    target_mean= np.mean(dataframe[target_variable])
    error= RSE/target_mean
    print(f"The intercept is {lm.intercept_}\n")
    print(f"The coefs are: {lm.coef_}\n")
    print(f"The R2 is: {lm.score(X,Y)}\n")
    print(f"The SSD is: {SSD}\n")
    print(f"The RSE is: {RSE}\n")
    print(f"The {target_variable} mean is: {target_mean}\n")
    print(f"The error is: {error}\n")
   

In [ ]:
polynomial_model(data,feature_cols,"price",3)

# We got an R2 of 92% and an error of 12%. Which is good. But now, we will use a func that increases the degrees to see wich one is better

In [ ]:
def degree_increase(dataframe,feature_cols,target_variable,Max_Degree_Number):
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn import linear_model
    for d in range(2,Max_Degree_Number+1):
        poly= PolynomialFeatures(degree=d)
        X= poly.fit_transform(dataframe[feature_cols])
        Y= dataframe[target_variable]
        lm= linear_model.LinearRegression()
        lm.fit(X,Y)
        dataframe["Cuadratic_Linear_Predict"]=lm.predict(X)
        dataframe["Cuadratic_Linear_RSE"]= (dataframe[target_variable] - dataframe["Cuadratic_Linear_Predict"])**2
        SSD= sum(dataframe["Cuadratic_Linear_RSE"])
        RSE= np.sqrt(SSD/(len(dataframe)-len(feature_cols) -1 ))
        target_mean= np.mean(dataframe[target_variable])
        error= RSE/target_mean
        print(f"For the regression of degree {d}:\nThe R2 is {lm.score(X,Y)}")
        print(f"The intercept is: {lm.intercept_}")
        print(f"The coefs are: {lm.coef_}")
        print(f"The error is {error}")
        print(f"The SSD is {SSD}")
        print(f"The RSE is {RSE}")
        print()
        

**We will set a maximum degree of 6**

In [ ]:
degree_increase(data,feature_cols,"price",6)

* We see that 3 is the best degree. 
* This is my first Kaggle Notebook and i am super new to ML. So i would love some feedback. 
* Thx for Watching